# Gaussian Elimination

In [24]:
%load_ext Cython

The Cython extension is already loaded. To reload it, use:
  %reload_ext Cython


In [25]:
%%cython --annotate
import cython
import sys
import cmath
import numpy as np #not working
cimport numpy as np
@cython.cdivision(True)
@cython.boundscheck(False) #deactivates checking bounds
cpdef solver(list U,list v): 
    cdef double[:, :] A = np.array(U)
    cdef double[:] b = np.array(v)
    global m,w,f
    # Variable matrix/vector
    cdef float dum
    cdef int dum2
    cdef Py_ssize_t i,y,j,d,r
    cdef Py_ssize_t la=len(A)
    cdef float norm
    inf=False
    list_zeropivot=[]
    #cdef np.ndarray[np.double_t,ndim=1] l
    #cdef np.ndarray[np.double_t,ndim=1] X
    #print(h)
    cdef double[:] l=np.array([i for i in range(1,la+1)],dtype=np.double)
    #cdef double[:] l=np.array([i for i in range(1,m+1)]+[i for i in range(1,w+1)],dtype=np.float64)
    d=0
    
    # For each iteration, it goes through the matrix and d is incremented 
    while(d<la-1):
        dum=A[d+1][d]
        y=d+1
        
        # Partial Pivoting, i.e switching rows to obtain row containing max element  
        if(A[d][d]==0):
            for i in range(1,la-d):
                if A[d+1][d]<A[d+i][d]:
                    y=d+i
            #A[[d,y]]=A[[y,d]]
            #b[[d,y]]=b[[y,d]]
            #l[[d,y]]=l[[y,d]]
            for r in range(la):
                A[d][r],A[y][r]=A[y][r],A[d][r]
                #b[d][r],b[y][r]=b[y][r],b[d][r]
                #l[d][r],l[y][r]=l[y][r],l[d][r]
            l[d],l[y]=l[y],l[d]
            b[d],b[y]=b[y],b[d]
            if(A[d][d]==0):
                inf=True
                list_zeropivot.append(d)
                d+=1
                continue
        
        # Gaussian elimination 
        for j in range(d+1,la):
            norm= A[j][d] / A[d][d]
            for i in range(d,la):
                A[j][i]-=A[d][i]*norm
            b[j]-=b[d]*norm
        d+=1
        
    # Normalisation of all elements wrt A(i,i)
    for j in range(0,la):
        norm=A[j][j]
        if norm==0:
            inf=True
            list_zeropivot.append(j)
            
        if norm:
            for i in range(len(A[j])):
                A[j][i]=A[j][i]/norm
            b[j] /= norm
    
    # Backsubstitution of values
    cdef double[:] X = np.zeros((la))
    X[la-1]=b[la-1]
    for i in range(2,la+1):
        dum2=i-1
        X[la-i]=b[la-i]
        while(dum2>0):
            X[la-i]-=A[la-i][la-dum2]*X[la-dum2]
            dum2-=1
    #print(X)
    # Checking for no/infinite solns
    for item in list_zeropivot:
        if X[item]!=0:
            return "Error! No solution exists!"
    if inf:
        return "Error! Infinite number of solutions exist!"
    
    return(X)

In [26]:
U=np.array(np.random.rand(10,10)).astype(float).tolist()
v=np.array(np.random.rand(10)).astype(float).tolist()
print(np.asarray(solver(U,v)))
%timeit solver(U,v)

[-1.08472664  1.22603913  2.61700433 -3.10056864 -3.00876806 -0.01622377
  2.61856797 -0.26594575  0.86605127  1.40514175]
8.49 µs ± 242 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


* **The time taken to solve a random 10×10 matrix has been brought down to below 10$\mu$s from about 80$\mu$s with the help of Cython.**

### Function for Gaussian Elimination
* **The function solver(A,b) takes two matrices A and b as its arguments and returns the matrix X, the solution for the equation AX=b. It prints accordingly if no/infinite solutions exist.**
* **The array l contains the variables involved in the equations and d is the number of times the whole matrix A was iterated through.**
* **Partial pivoting, i.e if the pivot turns out to be zero, we switch rows with the row($y^{th}$ row) containing the maximum element in that column.**
* **If the pivot remains zero, that means the system either has no solution or infinite number of solutions.** 
* **If the corresponding variable for that row turns out to be non-zero for any such case, then no solution exists. If it turns out to be zero for every such occurrence then the system has infinte number of solutions.** 
* **Once the pivot is fixed for that iteration, gaussian elimination is done, i.e every row below it is scaled and subtracted such that the elements below the pivot in that column become zero. This process is repeated till the last row is reached.**
* **The above row operations on matrix A are simultaneously performed on b as well, to ensure that the equation remains the same.**
* **The elements are then normalised with respect to the pivot in each row, i.e the A[i][i] element for each $i^{th}$ row.**
* **The required matrix X is then obtained by backsubstituting the values obtained from the bottom and solving the equations.**





In [27]:
%timeit np.linalg.solve(U,v)
print(np.linalg.solve(U,v))
#%timeit np.linalg.solve(np.array(A).astype(float).tolist(),np.array(b).astype(float).tolist())
#%timeit (np.linalg.solve(np.array(A).astype(float),np.array(b).astype(float)))


26 µs ± 338 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)
[-1.08472212  1.22603366  2.61701206 -3.10058189 -3.00877898 -0.01621497
  2.61857789 -0.26594545  0.86604483  1.40514707]


### Time Comparison
**The user-built function solver(A,b) takes around 150$\mu$s and the linalg.solve() takes around 20$\mu$s to compute the matrix X for a random 10×10 matrix A, which confirms that this is a fairly efficient way.**

# CIRCUIT ANALYSIS OF SPICE NETLISTS

In [28]:
%load_ext Cython

The Cython extension is already loaded. To reload it, use:
  %reload_ext Cython


### Data Extraction
* **The required netlist file is opened in read mode and every line is read into a list as an element.**
* **If nodes are represented as $n_i$, the character 'n' is removed using lstrip() method.**

In [29]:
%%cython --annotate
import cython
import sys
from cpython.string cimport PyString_AsString
import cmath
import numpy as np #not working
cimport numpy as np
global circ
with open('ckt2.netlist','r') as f:
    circ= f.readlines()
circ=[(i.rstrip('\n')) for i in circ]

### Separation of different components
* **Different lists are created for every components and sources and the component specifications are added to the corresponding lists.**
* **The comments or data(except AC frequency) before '.circuit' and after '.end' are discarded.**
* **The AC frequency is stored in the variable f, and m contains the number of nodes.**

### Magnitude and Phase
* **The functions get_mag() and get_angle() return the magnitude and phase of the given complex argument.**
* **The final answer is printed in the form of A cos(wt+$\phi$).**
* **This program can be extended to more than 1 frequency, but due to time constraints, has been limited to sources with maximum of one common AC frequency with few other DC sources, done with help of superposition**. 

In [30]:
%load_ext Cython

The Cython extension is already loaded. To reload it, use:
  %reload_ext Cython


**The Cython extension is loaded with the help of the above statement.**

In [31]:
%%cython --annotate
import cython
import sys
import cmath
import numpy as np #not working
cimport numpy as np
@cython.cdivision(True)
@cython.boundscheck(False) #deactivates checking bounds
cpdef solver(list U,list v): 
    cdef double[:, :] A = np.array(U)
    cdef double[:] b = np.array(v)
    global m,w,f
    # Variable matrix/vector
    cdef float dum
    cdef int dum2
    cdef Py_ssize_t i,y,j,d,r
    cdef Py_ssize_t la=len(A)
    cdef float norm
    #cdef np.ndarray[np.double_t,ndim=1] l
    #cdef np.ndarray[np.double_t,ndim=1] X
    #print(h)
    cdef double[:] l=np.array([i for i in range(1,m+1)]+[i for i in range(1,w+1)],dtype=np.float64)
    d=0
    
    # For each iteration, it goes through the matrix and d is incremented 
    while(d<la-1):
        dum=A[d+1][d]
        y=d+1
        
        # Partial Pivoting, i.e switching rows to obtain row containing max element  
        if(A[d][d]==0):
            for i in range(1,la-d):
                if A[d+1][d]<A[d+i][d]:
                    y=d+i
            #A[[d,y]]=A[[y,d]]
            #b[[d,y]]=b[[y,d]]
            #l[[d,y]]=l[[y,d]]
            for r in range(la):
                A[d][r],A[y][r]=A[y][r],A[d][r]
                #b[d][r],b[y][r]=b[y][r],b[d][r]
                #l[d][r],l[y][r]=l[y][r],l[d][r]
            l[d],l[y]=l[y],l[d]
            b[d],b[y]=b[y],b[d]
        
        # Gaussian elimination 
        for j in range(d+1,la):
            norm= A[j][d] / A[d][d]
            for i in range(d,la):
                A[j][i]-=A[d][i]*norm
            b[j]-=b[d]*norm
        d+=1
        
    # Normalisation of all elements wrt A(i,i)
    for j in range(0,la):
        norm=A[j][j]
        for i in range(len(A[j])):
            A[j][i]=A[j][i]/norm
        b[j] /= norm
    
    # Backsubstitution of values
    cdef double[:] X = np.zeros((la))
    X[la-1]=b[la-1]
    for i in range(2,la+1):
        dum2=i-1
        X[la-i]=b[la-i]
        while(dum2>0):
            X[la-i]-=A[la-i][la-dum2]*X[la-dum2]
            dum2-=1
    #print(X)
    
    # Converting rectangular form to phasor form
    mag=get_magp(X)
    #print(mag)
    #print("Magnitude and phase: ",mag)

cpdef spice(circ):
    cdef list R=[]
    #global Va,Vd
    cdef list Va=[]
    #Va=[]
    cdef list Vd=[]
    cdef list Ia=[]
    cdef list Id=[]
    cdef list L=[]
    cdef list C=[]
    cdef list l
    global m
    m=0
    cdef int lastd=0
    global f 
    f=np.exp(-100)
    cdef dict o,e
    o={'R':['resistor',R],'L':['inductor',L],'C':['capacitor',C]}
    e={'Vac':['ac voltage',Va],'Vdc':['dc voltage',Vd],'Idc':['dc current',Id],'Iac':['ac current',Ia]}
    cdef str key
    for i in circ:

        l=i.split()
        if l==['.circuit']:
            read=True
            continue
        if l[0]=='.ac':
            if lastd==1 and float(l[2])*2*np.pi!=f:
                sys.exit("ERROR! Not allowed to input 2 different frequencies!")
            f=float(l[2])*2*np.pi
            #print(f)
            lastd=1
            continue
        if l==['.end']:
            read=False
            continue
        #cdef str key
        for key in o:
            if l[0][0]== key and read:
                l[1]=l[1].lstrip('n')
                l[2]=l[2].lstrip('n')
                #print(f"Found a {d[key][0]} with value: ",float(l[3]))
                o[key][1].append(l)
                if l[1] != 'GND':
                    m=max(m,int(l[1]))
                if l[2] != 'GND':
                    m=max(m,int(l[2]))
                break

        for key in e:
            if l[0][0]+l[3] == key and read:
                l[1]=l[1].lstrip('n')
                l[2]=l[2].lstrip('n')
                #print(f"Found a/an {e[key][0]} source with value: ", float(l[4]))
                e[key][1].append(l)
                break
    cdef list c=R+L+C+Id+Ia+Vd+Va
    cdef int length=m+len(Va)+len(Vd)
    global w
    w=len(Va)+len(Vd)
    #print(length)
    cdef list A,b
    cdef int l1,l2
    cdef float l4
    #cdef double[:, :] A = np.array(U)
    #cdef double[:] b = np.array(v)
    for i in [Va+Ia,Vd+Id]:
        if(len(i)>0):
            row=m
            #A= np.zeros([length,length],dtype=complex)
            #b= np.zeros(length,dtype=complex)
            A = [length*[0] for i in range(length)]
            b=length*[0]
            count=0
            if i==Va+Ia:
                q=[1,1,0,0]
            elif i==Vd+Id:
                q=[0,0,1,1]
                f=np.exp(-500)

            fac={'R':1,'L':-complex(0,1/f),'C':complex(0,f)}
            for l in c:
                for key in fac:
                    if l[0][0]== key and key!='C':
                        dum=0
                        x=fac[key]
                        for j in range(1,3):
                            if l[j]!='GND':
                                d=int(l[j])
                                A[d-1][d-1]+=x/float(l[3])
                                dum+=1
                            if (dum==2):
                                A[d-1][int(l[j-1])-1]+=-x/float(l[3])
                                A[int(l[j-1])-1][d-1]+=-x/float(l[3])
                        break
                    elif l[0][0]== key and key=='C':
                        dum=0
                        x=fac[key]
                        for i in range(1,3):
                            if l[i]!='GND':
                                d=int(l[i])
                                A[d-1][d-1]+=x*float(l[3])
                                dum+=1
                            if (dum==2):
                                A[d-1][int(l[i-1])-1]+=-x*float(l[3])
                                A[int(l[i-1])-1][d-1]+=-x*float(l[3])
                        break
                
                
                
                
                if l[0][0]+l[3]== 'Iac':
                    if l[1]!= 'GND':
                        l1=int(l[1])
                        l4=float(l[4])
                        b[l1-1]+=-q[1]*l4
                    if l[2]!= 'GND':
                        l2=int(l[2])
                        l4=float(l[4])
                        b[l2-1]+=q[1]*l4

                if l[0][0]+l[3]== 'Idc':
                    if l[1]!= 'GND':
                        l1=int(l[1])
                        l4=float(l[4])
                        b[l1-1]+=-q[3]*l4
                    if l[2]!= 'GND':
                        l2=int(l[2])
                        l4=float(l[4])
                        b[l2-1]+=q[3]*l4


                if l[0][0]+l[3]== 'Vdc':
                    if l[1]!='GND':
                        l1=int(l[1])
                        A[row][l1-1]+=1
                        A[l1-1][row]+=1
                    if l[2]!='GND':
                        l2=int(l[2])
                        A[row][l2-1]+=-1
                        A[l2-1][row]+=-1
                    l4=float(l[4])
                    b[row]+=q[2]*l4
                    row+=1

                if l[0][0]+l[3]== 'Vac':
                    if l[1]!='GND':
                        l1=int(l[1])
                        A[row][l1-1]+=1
                        A[l1-1][row]+=1
                    if l[2]!='GND':
                        l2=int(l[2])
                        A[row][l2-1]+=-1
                        A[l2-1][row]+=-1
                    l4=float(l[4])
                    b[row]+=q[0]*l4
                    row+=1
            #solver(np.array(A).astype(complex).tolist(),np.array(b).astype(complex).tolist()) 
            if i==Vd+Id:
                f=0
            #print(A,b)
            #print("Frequency =",f)
            
            solver(A,b) 

            #print(A,b)
            
cpdef get_magp(z):
    # magnitude
    cdef list l=[]
    mag = np.abs(z).tolist()
    # phase angle in radians
    ang = np.angle(z).tolist()
    cdef Py_ssize_t lz=len(z)
    cdef Py_ssize_t i
    for i in range(lz):
        l.append("["+(str(mag[i])+"]cos("+str(float(f))+"t+("+str(ang[i])+") )"))
    return l    
#solver(np.array(A).astype(complex),np.array(b).astype(complex))

* **The required libraries and packages are imported and the required decorators are used.**
* **A white line means that the code generated doesn’t interact with Python, so will run as fast as normal C code. The darker the yellow, the more Python interaction there is in that piece of code. Those yellow lines will usually operate on Python objects or do other kinds of higher-level operations than what can easily be translated into simple and fast C code.**
* **The function declaration and return use the Python interpreter so it makes sense for those lines to be yellow. The same goes for the list comprehension because it involves the creation of a Python object.**
* **Since both C and python variables are used simultaneously, type-conversion might consume time; which can be minimised with the help of memoryviews.**
* **The statement `cdef double[:, :] A = np.array(U)` enables memoryviews, which are C structures that can hold a pointer to the data of a Numpy array and all the necessary buffer metadata to provide efficient and safe access. They also support slices and can be indexed by C integers, thus allowing fast access to the Numpy array data.**
* **The "cdef" keyword, i.e. `cdef <datatype> <var>` is also used within functions to assign datatype to variables. It can only be used at the top indentation level, i.e. globally or inside the outermost function; which is why this can't be used in cases where a new data structure is generated for each iteration inside some function.**
* **Python and C have different scoping rules. Cython uses the same scoping rules as Python so variables "declared" inside of a for /if/ while or other block are in scope for the whole function. This is also true for variables declared using `cdef`, but as you've seen these variable must be declared at the function level and not in a sub-block.**
* **Py_ssize_t is the proper C type, used for python array indices, for eg, the statement `cdef Py_ssize_t la,i,d` speeds up the for loop iteration `for i in range(1,la-d):`.**
* **The keyword `cdef` can be used for assigning C-exclusive datatypes and functions and `cpdef` for general pythonic functions.**
* **There is a slight change in the part of code that switches two rows of an array. The initial code was written for numpy arrays and the new code has been written for general python lists.**
* **The speeding up process using Cython has been emphasized with respect to the gaussian elimination,i.e. `solver()` function, and can be extended to the entire simulator by using lists with strings,fast string arrays, which might increase the complexity level of the program.**
* **The data preprocessing code can also be written in a more efficient and friendly way for cython to be implemented effectively.**
* **The time taken by the entire simulator function has been brought down to around 100$\mu$s from 160$\mu$s, by the optimizations brought about in the `solver()` function.** 

In [32]:
%timeit (spice(circ))

103 µs ± 2.45 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Function for Gaussian Elimination
* **The function solver(A,b) takes two matrices A and b as its arguments and returns the matrix X, the solution for the equation AX=b. It prints accordingly if no/infinite solutions exist.**
* **The array l contains the variables involved in the equations and d is the number of times the whole matrix A was iterated through.**
* **Partial pivoting, i.e if the pivot turns out to be zero, we switch rows with the row($y^{th}$ row) containing the maximum element in that column.**
* **If the pivot remains zero, that means the system either has no solution or infinite number of solutions.** 
* **If the corresponding variable for that row turns out to be non-zero for any such case, then no solution exists. If it turns out to be zero for every such occurrence then the system has infinte number of solutions.** 
* **Once the pivot is fixed for that iteration, gaussian elimination is done, i.e every row below it is scaled and subtracted such that the elements below the pivot in that column become zero. This process is repeated till the last row is reached.**
* **The above row operations on matrix A are simultaneously performed on b as well, to ensure that the equation remains the same.**
* **The elements are then normalised with respect to the pivot in each row, i.e the A[i][i] element for each $i^{th}$ row.**
* **The required matrix X is then obtained by backsubstituting the values obtained from the bottom and solving the equations.**



### Constructing A and b ( Nodal Analysis )
* **If AC and DC sources are both present (or different frequencies) , we use superposition to separate the ac and dc sources   (different frequency sources) and add the final solution matrix**.
* **If a resistor R is found to be connected across nodes i and j, $\frac{1}{R}$ is added to A[i][i] and A[j][j] and -$\frac{1}{R}$ is added to A[i][j] and A[j][i]. If a resistor connected to the ground, it is modified accordingly.**
* **The same process is carried out for inductors and capacitors, with a small change in their impedances. For inductors, -$\frac{j}{\omega L}$ is added and for capacitors, j$\omega$C is added.**
* **For Voltage source connected across nodes i and j, 1 is added to A[row][i] and -1 is added to A[row][j] and the corresponding value of V is added to b[row].**
* **For Current source connected across nodes i and j, I is added to b[i] and -I added to b[j].**
* **For sources with one terminal connected to ground, changes are done accordingly.**
* **The matrices A and b are now given to function solver() to solve for the matrix X.**
* **The matrix X contains the node voltages in numerical order and then the currents through the dc and ac sources (internally arranged in numerical order) respectively.**
* **The two final solution matrices, for AC part and DC part respectively, done by superposition has been printed out in the same order as a function of time.**